<a href="https://colab.research.google.com/github/parisa-kavian/AI-Plan-Generator/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install langchain openai langchain_community

In [3]:
import os

os.environ['OPENAI_API_KEY'] = "sk-proj-G4OwU0k5Czohsr6kFmwrZDOIq45oy3Zm2K9PBA-####"

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, PositiveInt, PositiveFloat, field_validator

# User Information
class UserProfile(BaseModel):
    age: PositiveInt
    gender: str
    weight: PositiveFloat  # kg
    height: PositiveFloat  # cm
    freetime: PositiveInt  # minutes per day
    goal: str
    activity_level: str
    dietary_preferences: str = None  # Optional

    @field_validator("gender")
    @classmethod
    def validate_gender(cls, v: str) -> str:
        if v.lower() not in ["man", "woman", "non-binary", "other"]:
            raise ValueError("Gender must be valid")
        return v.lower()

    def bmi(self):
        return round(self.weight / ((self.height / 100) ** 2), 1)

    def bmr(self):
        if self.gender == "man":
            return round(10 * self.weight + 6.25 * self.height - 5 * self.age + 5, 1)
        elif self.gender == "woman":
            return round(10 * self.weight + 6.25 * self.height - 5 * self.age - 161, 1)
        else:
            return round(10 * self.weight + 6.25 * self.height - 5 * self.age, 1)


# Plan
def create_prompt(profile: UserProfile, task: str) -> str:
    """Creates the prompt based on the user's complete profile and a specified task."""
    dietary_info = f"Dietary preferences: {profile.dietary_preferences}\n" if profile.dietary_preferences else ""

    return f"""
You are a professional expert in {task}.
Create a weekly {task} plan for a {profile.age}-year-old {profile.gender} with the following profile:
Weight: {profile.weight} kg | Height: {profile.height} cm | BMI: {profile.bmi()} | BMR: {profile.bmr()}
Activity Level: {profile.activity_level}
Daily free time: {profile.freetime} minutes
Goal: {profile.goal}
{dietary_info}
Provide a detailed day-by-day plan for 6 days.
"""


# LLM setup
llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory)


def generate_plan(profile: UserProfile, task: str) -> str:
    """Generates the plan using the prompt and the language model."""
    prompt = create_prompt(profile, task)
    return conversation.run(prompt)


# Interactive Program
if __name__ == "__main__":

    print("Welcome to the personalized plan generator!")
    print("---------------------------------------")

    # Get user information
    try:
        age = int(input("Please enter your age: "))
        gender = input("Please enter your gender (man/woman): ")
        weight = float(input("Please enter your weight (in kg): "))
        height = float(input("Please enter your height (in cm): "))
        freetime = int(input("Please enter your daily free time for exercise (in minutes): "))
        goal = input("Please enter your goal (e.g., build muscle, lose weight, maintain health): ")
        activity_level = input("Please enter your activity level (e.g., sedentary, lightly active, active): ")
        dietary_preferences = input("Enter your dietary preferences or restrictions (optional, e.g., vegetarian): ") or None

        user = UserProfile(
            age=age,
            gender=gender,
            weight=weight,
            height=height,
            freetime=freetime,
            goal=goal,
            activity_level=activity_level,
            dietary_preferences=dietary_preferences
        )

        # Choose the type of plan
        task_choice = input("What kind of plan do you want? (meal planning or workout plan): ")

        if task_choice.lower() not in ["meal planning", "workout plan"]:
            print("Invalid choice. The program will default to 'meal planning'.")
            task = "meal planning"
        else:
            task = task_choice

        print("\nGenerating plan...")

        plan = generate_plan(user, task=task)
        print("\n--- Generated Plan ---")
        print(plan)

    except ValueError as e:
        print(f"Invalid input: {e}. Please make sure you enter numbers for age, weight, height, and freetime.")
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please ensure your API key is correct and your internet connection is stable.")

/tmp/ipython-input-194180574.py:54: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
/tmp/ipython-input-194180574.py:55: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/tmp/ipython-input-194180574.py:56: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(llm=llm, memory=memory)


Welcome to the personalized plan generator!
---------------------------------------
What kind of plan do you want? (meal planning or workout plan): meal planning

Generating plan...


/tmp/ipython-input-194180574.py:62: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return conversation.run(prompt)



--- Generated Plan ---
Certainly! Here is a detailed day-by-day meal planning plan for a 29-year-old woman with the provided profile:

Day 1:
- Breakfast: Greek yogurt with mixed berries and almonds
- Snack: Carrot sticks with hummus
- Lunch: Quinoa salad with grilled chicken and avocado
- Snack: Apple slices with peanut butter
- Dinner: Baked salmon with roasted vegetables
- Dessert: Dark chocolate squares

Day 2:
- Breakfast: Oatmeal with sliced bananas and walnuts
- Snack: Greek yogurt with honey
- Lunch: Whole grain wrap with turkey, lettuce, tomato, and avocado
- Snack: Mixed nuts
- Dinner: Stir-fried tofu with broccoli and brown rice
- Dessert: Fresh fruit salad

Day 3:
- Breakfast: Whole grain toast with avocado and poached eggs
- Snack: Cottage cheese with pineapple chunks
- Lunch: Lentil soup with a side of quinoa
- Snack: Rice cakes with almond butter
- Dinner: Grilled shrimp with zucchini noodles
- Dessert: Chia seed pudding with berries

Day 4:
- Breakfast: Smoothie with s